## Import requirements

In [19]:
import yaml, time, json, traceback
from collections import defaultdict
from typing import Dict, List
from pprint import PrettyPrinter
from tabulate import tabulate
import networkx as nx
from itertools import count
from fabrictestbed_extensions.fablib.fablib import fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

Define a better printer for debugging process

In [20]:
def pretty_print_defaultdict(d):
    # Create PrettyPrinter instance with custom formatting
    pp = PrettyPrinter(indent=4, width=80, sort_dicts=True)
    
    # Convert defaultdict to dict before printing
    pp.pprint(dict(d))

## Get values from YAML file

In [13]:
with open('./config.yaml', 'r') as config_file:
    base_config = yaml.safe_load(config_file)
    
client_node_names: List[str] = base_config['topology']['client_nodes']
client_connections: Dict[str, List[str]] = base_config['topology']['client_connections']
general_node_names: List[str] = base_config['topology']['nodes']
general_connections: Dict[str, List[str]] = base_config['topology']['connections']

all_nodes = client_node_names + general_node_names
node_amount = len(all_nodes)

print(f'Client node names: {client_node_names}')
print(f'\nClient connections: {json.dumps(client_connections, indent=2)}')
print(f'\nGeneral node names: {general_node_names}')
print(f'\nGeneral connections: {json.dumps(general_connections, indent=2)}')

Client node names: ['node1', 'node6', 'node11']

Client connections: {
  "node1": [
    "node2"
  ],
  "node6": [
    "node7"
  ],
  "node11": [
    "node12"
  ]
}

General node names: ['node2', 'node3', 'node4', 'node5', 'node7', 'node8', 'node9', 'node10', 'node12', 'node13', 'node14', 'node15', 'node16', 'node17', 'node18', 'node19', 'node20']

General connections: {
  "node2": [
    "node3",
    "node4"
  ],
  "node3": [
    "node2",
    "node5",
    "node7"
  ],
  "node4": [
    "node2",
    "node8",
    "node9"
  ],
  "node5": [
    "node3",
    "node10"
  ],
  "node7": [
    "node3",
    "node8",
    "node13"
  ],
  "node8": [
    "node4",
    "node7",
    "node14"
  ],
  "node9": [
    "node4",
    "node10",
    "node15"
  ],
  "node10": [
    "node5",
    "node9",
    "node16"
  ],
  "node12": [
    "node13",
    "node16"
  ],
  "node13": [
    "node7",
    "node12",
    "node17"
  ],
  "node14": [
    "node8",
    "node15",
    "node18"
  ],
  "node15": [
    "node9",
    "no

## Define global variables

In [14]:
manager = fablib_manager()

time_stamp = int(time.time())
experiment_slice = fablib.new_slice(name=f"hydra_auto_{time_stamp}")

# Basic node configs
experiment_cores = 4
experiment_ram = 8
experiment_disk = 20
experiment_image = "default_ubuntu_20"

# In case of fail, pick manually
def filter_sites(site):    
    try:
        if (site["state"] == "Active" and 
            site["hosts"] > 0 and 
            site["cores_available"] >= experiment_cores * 2 and 
            site["ram_available"] >= experiment_ram * 2 and 
            site["disk_available"] >= experiment_disk * 2
        ):
            return True
        return False
    except Exception as error:
        print(f"Filter error for site {site}: {error}")
        return False

# Change amount of site based on the topology size
experiment_sites = manager.get_random_sites(
    count=4,
    avoid=['NEWY'],
    filter_function=filter_sites,
    update=True,
    unique=True 
)

print(f'Available sites: {experiment_sites}')

ConfigException: Token file does not exist, please provide the token at location: /Users/gportdev/.tokens.json!

## Add nodes and interfaces

In [15]:
site_selector = count()

nodes = defaultdict(dict)
nodes_interfaces = defaultdict(list)
client_nodes = defaultdict(dict)
client_interfaces = defaultdict(list)
networks = defaultdict(list)

# Helper function to count total connections for a node
def count_total_connections(node_name):
    # For client nodes, just count their outgoing connections
    if node_name in client_node_names:
        return len(client_connections.get(node_name, []))
    
    # For general nodes, count unique connections
    connections = set()
    
    # Add outgoing connections
    for target in general_connections.get(node_name, []):
        connections.add(tuple(sorted([node_name, target])))
    
    # Add incoming connections
    for source, targets in general_connections.items():
        if node_name in targets:
            connections.add(tuple(sorted([source, node_name])))
    
    # Add connections from clients
    for client, targets in client_connections.items():
        if node_name in targets:
            connections.add(tuple(sorted([client, node_name])))
            
    return len(connections)

try:
    # Add each client node to the slice
    for node_name in client_node_names:
        print(f'> Processing client {node_name}')
        current_node = experiment_slice.add_node(
            name=node_name,
            site=experiment_sites[next(site_selector) % len(experiment_sites)],
            cores=experiment_cores,
            ram=experiment_ram,
            disk=experiment_disk,
            image=experiment_image
        ) 
        
        # Calculate total needed interfaces for this client
        total_interfaces = count_total_connections(node_name)
        print(f'> {node_name} needs {total_interfaces} interfaces')
        
        # Add all needed interfaces to the node
        for i in range(total_interfaces):
            print(f'> \tAdding interface {i+1} to client {node_name}')
            new_connection = current_node.add_component(
                model='NIC_Basic', 
                name=f'nic-{node_name[-1]}-{i+1}'
            ).get_interfaces()[0]
            
            client_interfaces[node_name].append(new_connection)
        
        client_nodes[node_name] = current_node
        
    # Add each general node to the slice
    for node_name in general_node_names:
        print(f'> Processing {node_name}')
        
        current_node = experiment_slice.add_node(
            name=node_name,
            site=experiment_sites[next(site_selector) % len(experiment_sites)],
            cores=experiment_cores,
            ram=experiment_ram,
            disk=experiment_disk,
            image=experiment_image
        )
        
        # Calculate total needed interfaces for this node
        total_interfaces = count_total_connections(node_name)
        print(f'> {node_name} needs {total_interfaces} interfaces')
        
        # Add all needed interfaces to the node
        for i in range(total_interfaces):
            print(f'> \tAdding interface {i+1} to {node_name}')
            new_connection = current_node.add_component(
                model='NIC_Basic',
                name=f'nic-{node_name[-1]}-{i+1}'
            ).get_interfaces()[0]
            
            nodes_interfaces[node_name].append(new_connection)
            
        nodes[node_name] = current_node

    # Debug output
    print("\nInterface counts per node:")
    for node_name in client_nodes:
        print(f"{node_name}: {len(client_interfaces[node_name])} interfaces")
    for node_name in nodes:
        print(f"{node_name}: {len(nodes_interfaces[node_name])} interfaces")
    
    print("\nConnection counts per node:")
    for node_name in {**client_nodes, **nodes}:
        print(f"{node_name}: {count_total_connections(node_name)} connections")
    
except Exception as e:
    print(f"> Node configuration failed: \n{e}")
    traceback.print_exc(e)

> Processing client node1
Fail: name 'experiment_slice' is not defined


## Add Networks

In [ ]:
# Create copies of interface tables
client_interfaces_copy = {k: v[:] for k, v in client_interfaces.items()}
nodes_interfaces_copy = {k: v[:] for k, v in nodes_interfaces.items()}

# Helper function to check if nodes are already connected
def are_nodes_connected(node1, node2, networks):
    return any(
        f'net-{n1}-{n2}' in networks.get(node1, [])
        for n1 in [node1[-1], node2[-1]]
        for n2 in [node1[-1], node2[-1]]
    )

# Helper function to create network connection
def create_network_connection(source_node, target_node, source_interface, target_interface, networks):
    network_name = f'net-{source_node[-1]}-{target_node[-1]}'
    
    # Create L2 network
    experiment_slice.add_l2network(
        name=network_name,
        interfaces=[source_interface, target_interface]
    )
    
    # Record network connection
    networks[source_node].append(network_name)
    networks[target_node].append(network_name)

try:
    # --- Add networks for clients
    for node_name in client_node_names:
        for connection in client_connections[node_name]:
            # Check if these nodes are not already connected
            if not are_nodes_connected(node_name, connection, networks):
                
                # Check if interfaces are available for both nodes
                if (node_name in client_interfaces_copy and client_interfaces_copy[node_name] and
                    connection in nodes_interfaces_copy and nodes_interfaces_copy[connection]):
                    
                    # Get interfaces for the connection
                    source_interface = client_interfaces_copy[node_name][0]
                    target_interface = nodes_interfaces_copy[connection][0]
                    
                    create_network_connection(
                        node_name, connection, 
                        source_interface, target_interface, 
                        networks
                    )
                    
                    # Remove used interfaces from copies
                    client_interfaces_copy[node_name].remove(source_interface)
                    nodes_interfaces_copy[connection].remove(target_interface)

    # --- Add networks for general nodes
    # Process all connections from general_connections
    processed_pairs = set()
    
    for source_node, target_nodes in general_connections.items():
        for target_node in target_nodes:
            # Create a sorted pair to avoid duplicates
            node_pair = tuple(sorted([source_node, target_node]))
            
            # Skip if this pair has been processed
            if node_pair in processed_pairs:
                continue
                
            # Check if these nodes are not already connected
            if not are_nodes_connected(source_node, target_node, networks):
                
                # Check if interfaces are available for both nodes
                if (source_node in nodes_interfaces_copy and nodes_interfaces_copy[source_node] and
                    target_node in nodes_interfaces_copy and nodes_interfaces_copy[target_node]):
                    
                    # Get interfaces for the connection
                    source_interface = nodes_interfaces_copy[source_node][0]
                    target_interface = nodes_interfaces_copy[target_node][0]
                    
                    create_network_connection(
                        source_node, target_node, 
                        source_interface, target_interface, 
                        networks
                    )
                    
                    # Remove used interfaces from copies
                    nodes_interfaces_copy[source_node].remove(source_interface)
                    nodes_interfaces_copy[target_node].remove(target_interface)
            
            # Mark this pair as processed
            processed_pairs.add(node_pair)

except Exception as e:
    traceback.print_exc(e)

# Print debug information
print("Final networks:", dict(networks))
print("Remaining interfaces for nodes:", dict(nodes_interfaces_copy))
print("Remaining interfaces for clients:", dict(client_interfaces_copy))

## Print information about nodes and connections

In [ ]:
print(f'Nodes above. {pretty_print_defaultdict(nodes)}\n')
print(f'Nodes interfaces above. {pretty_print_defaultdict(nodes_interfaces)}\n')
print(f'Client nodes above. {pretty_print_defaultdict(client_nodes)}\n')
print(f'Client nodes interfaces above. {pretty_print_defaultdict(client_interfaces)}\n')
print(f'Networks above. {pretty_print_defaultdict(networks)}\n')

## Submit slice (Wait until Stable)

In [ ]:
# Create a checkpoint and only execute the rest of the code if the slice is stable
try:
    experiment_slice.submit()
except Exception as e:
    print(f"Slice submission error: {e}")

## Install Hydra dependencies in each node

In [ ]:
# For each node, install all the dependencies

try:
    for node in experiment_slice.get_nodes():
        stdout, stderr = node.execute('sudo apt update && sudo apt upgrade -y')
        stdout, stderr = node.execute('sudo apt install software-properties-common')
        stdout, stderr = node.execute('sudo add-apt-repository ppa:named-data/ppa')              
        stdout, stderr = node.execute('sudo apt update')
        stdout, stderr = node.execute('sudo apt install nfd')
        stdout, stderr = node.execute('sudo apt -y install python3-pip libndn-cxx-dev ndnping ndnpeek ndn-dissect ndnchunks ndnsec')
        stdout, stderr = node.execute('pip3 install packing')
        stdout, stderr = node.execute('git clone https://github.com/tntech-ngin/ndn-hydra.git')
        stdout, stderr = node.execute('cd ndn-hydra/ && pip install -e .')
        stdout, stderr = node.execute('pip3 install python-ndn ndn-storage ndn-svs numpy')
        stdout, stderr = node.execute('sudo apt install net-tools')
        stdout, stderr = node.execute('sudo cp /etc/ndn/nfd.conf.sample /etc/ndn/nfd.conf')
except Exception as e:
    print(f"Exception: {e}")

## NFD environment configuration for each node

In [ ]:
# Bring up the interfaces

for node in experiment_slice.get_nodes():
    for interface in node.get_interfaces():
        stdout, stderr = node.execute(f'sudo ip link set dev {interface.get_device_name()} up')
        stdout, stderr = node.execute('sleep 2; ip a')

for i, node_name in enumerate(client_node_names):
    node = experiment_slice.get_node(node_name)
    stdout, stderr = node.execute('nfd-stop')
    stdout, stderr = node.execute('nfd-start')
    stdout, stderr = node.execute('sleep 2; nfd-status')
    stdout, stderr = node.execute(f'ndnsec key-gen /client{i} | ndnsec cert-install -')
    
for i, node_name in enumerate(general_node_names):
    node = experiment_slice.get_node(node_name)
    stdout, stderr = node.execute('nfd-stop')
    stdout, stderr = node.execute('nfd-start')
    stdout, stderr = node.execute('sleep 2; nfd-status')
    stdout, stderr = node.execute(f'ndnsec key-gen /hydra/{node_name} | ndnsec cert-install -')

## Configure NFD faces and routes

In [ ]:
client_interfaces_copy = defaultdict(list)

for client_node_name in client_node_names:
    client_node = experiment_slice.get_node(client_node_name)
    for interface in client_node.get_interfaces():
        client_interfaces_copy[client_node_name].append(interface)
        
nodes_interfaces_copy = defaultdict(list)
for node_name in nodes_interfaces_copy:
    node = experiment_slice.get_node(node_name)
    for interface in node.get_interfaces():
        nodes_interfaces_copy[node_name].append(interface)

nfd_connections = defaultdict(list)

def are_nodes_nfd_connected(source_node: str, target_node: str, all_nfd_connections: dict[str, list[str]]):
    return target_node in all_nfd_connections.get(source_node, [])

try:      
    for client_node_name in client_node_names:
        for connection in client_connections[client_node_name]:
            source_node = nodes_interfaces_copy[client_node_name]
            
            # Check if these nodes are not already connected
            if not are_nodes_nfd_connected(client_node_name, connection, nfd_connections):
                print(f'NFD Connecting {client_node_name} to {connection}...')
                
                # Check if interfaces are available for both nodes
                if (client_node_name in client_interfaces_copy and 
                        client_interfaces_copy[client_node_name] and 
                        connection in nodes_interfaces_copy and nodes_interfaces_copy[connection]):
                            
                    # Get interfaces for the connection
                    source_node_interface = client_interfaces_copy[client_node_name][0]
                    target_node_interface = nodes_interfaces_copy[connection][0]
                    
                    print(f'Source node interface: {source_node_interface}')
                    print(f'Target node interface: {target_node_interface}')
                    
                    source_interface_mac_address, local_nic_name = "ether://[" + source_node_interface.get_mac() + "]", "dev://" + target_node_interface.get_device_name()
            
                    stdout, stderr = source_node.execute(f'nfdc face create remote {source_interface_mac_address} local {local_nic_name}')
                    stdout, stderr = source_node.execute(f'nfdc route add / {source_interface_mac_address}')
                    stdout, stderr = source_node.execute(f'nfdc cs config capacity 0 admit off serve off')
                    
                    # Remove used interfaces from copies
                    client_interfaces_copy[client_node_name].remove(source_node_interface)
                    nodes_interfaces_copy[connection].remove(target_node_interface)
                    
except Exception as e:
    traceback.print_exc(e)
            
processed_nfd_pairs = set()
try:
    for source_node_name, target_nodes in general_connections.items():
        for target_node_name in target_nodes:
            print(f'NFD Connecting {source_node_name} to {target_node_name}...')
            
            # Create a sorted pair to avoid duplicates
            node_pair = tuple(sorted([source_node_name, target_node_name]))
            
            # Skip if this pair has been processed
            if node_pair in processed_nfd_pairs:
                continue
                
            # Check if these nodes are not already connected
            if not are_nodes_nfd_connected(source_node_name, target_node_name, nfd_connections):
                source_node = nodes_interfaces_copy[source_node_name]
               
                # Check if interfaces are available for both nodes
                if (source_node_name in nodes_interfaces_copy and nodes_interfaces_copy[source_node_name] and
                            target_node_name in nodes_interfaces_copy and nodes_interfaces_copy[target_node_name]):
                    
                    source_node_interface = nodes_interfaces_copy[source_node_name][0]
                    target_node_interface = nodes_interfaces_copy[target_node_name][0]
                    
                    print(f'Source node interface: {source_node_interface}')
                    print(f'Target node interface: {target_node_interface}')
                    
                    # Perform connections
                    stdout, stderr = source_node.execute('nfdc strategy set /hydra/group /localhost/nfd/strategy/multicast')
                    
                    remote_mac_address, local_nic_name = "ether://[" + source_node_interface.get_mac() + "]", "dev://" + target_node_interface.get_device_name()
                    
                    stdout, stderr = source_node.execute(f'nfdc face create remote {remote_mac_address} local {local_nic_name}')
                    stdout, stderr = source_node.execute(f'nfdc route add /hydra/{target_node_name} {remote_mac_address}')
                    stdout, stderr = source_node.execute(f'nfdc route add /{target_node_name} {remote_mac_address}')
                    stdout, stderr = source_node.execute(f'nfdc route add /hydra/group {remote_mac_address}')
                    
                    # Disable cache
                    stdout, stderr = source_node.execute(f'nfdc cs config capacity 0 admit off serve off')
                    
                    # Remove used interfaces from copies
                    nodes_interfaces_copy[source_node_name].remove(source_node_interface)
                    nodes_interfaces_copy[target_node_name].remove(target_node_interface)    
                    
                # Mark this pair as processed
                processed_nfd_pairs.add(node_pair)

except Exception as e:
    traceback.print_exc(e)

print(processed_nfd_pairs)